In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import joblib

from data_preprocessing import pipeline
from feature_preprocessing_EG import pipeline2

### 데이터 전처리

In [13]:
# 로우 데이터 불러오기
file_path = "./storage/raw_data/병합_청약매물_목록_정보_픽스2.csv"
df = pd.read_csv(file_path)

# 데이터 전처리
preprocessing_pipeline = pipeline()
df = preprocessing_pipeline.transform(df)

# 파일 저장
file_version = "250318-유진1"
output_file = f"./storage/train_data/train-{file_version}.csv"
df.to_csv(output_file, index=False, encoding='cp949')

C:\Users\user\AppData\Local\Temp\ipykernel_25248\659066341.py:3: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
c:\Users\user\mid-project\src\data_preprocessing.py:122: DtypeWarning: Columns (36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_estate_price = pd.read_csv(
c:\Users\user\mid-project\src\data_preprocessing.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['최저당첨가점','최고당첨가점', '평균당첨가점']].fillna(0, inplace=True)


In [14]:
df

,공급지역코드,공급규모,투기과열지구,조정대상지역,분양가상한제,정비사업,공공주택지구,대규모택지개발지구,수도권내민영공공주택지구,공급세대수,...,토픽 2,토픽 3,토픽 4,토픽 5,토픽 6,토픽 7,전용면적,공급금액(최고가 기준),법정동코드,전용면적당 시세차익
44,400,29,N,N,N,Y,N,N,N,5,...,0.001167,0.001200,0.001167,0.050333,0.001200,0.924133,59.9393,4.497000e+08,2.817710e+09,7.502304e+06
48,400,29,N,N,N,Y,N,N,N,4,...,0.001167,0.001200,0.001167,0.050333,0.001200,0.924133,74.9227,5.675000e+08,2.817710e+09,7.574187e+06
60,100,482,Y,Y,Y,Y,N,N,N,35,...,0.090867,0.001267,0.719333,0.002967,0.001267,0.037367,59.9800,1.724700e+09,1.165010e+09,2.875227e+07
64,100,482,Y,Y,Y,Y,N,N,N,30,...,0.090867,0.001267,0.719333,0.002967,0.001267,0.037367,59.9500,1.760600e+09,1.165010e+09,2.936549e+07
68,100,482,Y,Y,Y,Y,N,N,N,14,...,0.090867,0.001267,0.719333,0.002967,0.001267,0.037367,59.7800,1.796500e+09,1.165010e+09,3.004954e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17234,410,747,N,N,Y,N,Y,Y,Y,161,...,0.157967,0.000900,0.517567,0.168600,0.007433,0.085333,84.1316,4.252000e+08,4.139013e+09,5.053664e+06
17238,410,747,N,N,Y,N,Y,Y,Y,57,...,0.157967,0.000900,0.517567,0.168600,0.007433,0.085333,84.3852,4.231000e+08,4.139013e+09,5.013590e+06
17240,410,747,N,N,Y,N,Y,Y,Y,57,...,0.157967,0.000900,0.517567,0.168600,0.007433,0.085333,84.3852,4.231000e+08,4.139013e+09,5.013590e+06
17244,410,747,N,N,Y,N,Y,Y,Y,216,...,0.157967,0.000900,0.517567,0.168600,0.007433,0.085333,84.3832,4.272000e+08,4.139013e+09,5.062296e+06


### 모델 학습 및 평가

In [15]:
# 전처리된 데이터 가져오기

# file_version = "250317-01"
# file_path = f"./storage/train_data/t``ain-{file_version}.csv"
# df = pd.read_csv(file_path, encoding='cp949')

df = pd.read_csv("../src/storage/train_data/train-250318-유진1.csv", encoding='cp949')



In [ ]:
X = df.drop(columns=["최저당첨가점"])
Y = df["최저당첨가점"]

X_train, X_test, y_train, y_test = train_test_split(
    X,  # 타겟(최저당첨가점) 제외
    Y,  # 타겟
    test_size=0.2,
    random_state=42,
    stratify=df["투기과열지구"]  # 분포 맞추기 위해 미달여부로 stratify
)

In [19]:
feature_pipeline = pipeline2()

# 학습 데이터(X_train)에 fit
feature_pipeline.fit(X_train)

# 학습 데이터(X_train)를 변환
X_train_transformed = feature_pipeline.transform(X_train)

# 테스트 데이터(X_test)를 변환
X_test_transformed = feature_pipeline.transform(X_test)

KeyError: "['미달여부'] not in index"

In [12]:
# TODO: 여러 모델 학습 및 하이퍼파라미터 최적화

model = RandomForestRegressor(
    n_estimators=100,
    max_depth=None,
    random_state=42
)

# 모델 학습
model.fit(X_train_transformed, y_train)

# 예측
y_pred = model.predict(X_test_transformed)

# 평가 (RMSE와 R^2 계산)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse) # 낮을수록 좋음
print("R^2 Score:", r2) # 1에 가까울수록 좋음

RMSE: 11.967034707109335
R^2 Score: 0.6976762187474028


### 모델 저장 

In [7]:
# 모델 저장 및 로드

version = '0.0.1'

# 모델 저장
joblib.dump(model, f"./storage/trained_model/model_{version}.pkl")

['./storage/trained_model/model_0.0.1.pkl']

In [8]:
# 모델 로드
loaded_model = joblib.load("./storage/trained_model/model_0.0.1.pkl")

# 예측
# X_test = np.array([[1, 2]])
# prediction = loaded_model.predict(X_test)
# print("Prediction:", prediction)